<h1 align="center">Procesamiento de muestra de datos del Covid19 en Colombia.</h1>
<div align="right">David A. Miranda, PhD<br>2021</div>

En este Jypyter Notebook se analiza una muestra de datos de Covid19 extraída de [datos.org.co](https://www.datos.gov.co/Salud-y-Protecci-n-Social/Casos-positivos-de-COVID-19-en-Colombia/gt2j-8ykr/data).

## 1. Cargar Librerías

In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import seaborn as sns
from scipy import stats
from datetime import datetime

ModuleNotFoundError: No module named 'numpy'

## 2. Cargar datos
### 2.1. Carga datos locales, si tienes clonado el repositorio

In [ ]:
data = pd.DataFrame([])
try:
    data = pd.read_csv('../data/20210411_Covid19_Santander.cvs.zip', low_memory=False)
except:
    print('Error! There are no local data; try with online data.')

### 2.2. Carga datos desde la nube

In [ ]:
try:
    data = pd.read_csv('https://github.com/davidalejandromiranda/StatisticalPhysics/blob/main/data/20210411_Covid19_Santander.cvs?raw=true', low_memory=False, error_bad_lines=False)
except:
    print('Error! It was not possible to get online data; please, check your Internet connection.')

## 3. Descripción de los datos
### 3.1. El método *describe()* de pandas

In [ ]:
data.describe()

### 3.2. Columnas en la tabla de datos

In [ ]:
data.columns

### 3.3. Valores unicos de una columna

In [ ]:
data['Nombre municipio'].unique()

### 3.4. Obtención de todas las filas para un valor de una columna

In [ ]:
data_buc = data.loc[data['Nombre municipio'] == 'BUCARAMANGA']

In [ ]:
mask = data['Nombre municipio'] == 'BUCARAMANGA'
data_buc = data.loc[mask]

In [ ]:
data_buc.describe()

## 4. Análisis de la columna *Recuperado* para datos Covid19
### 4.1. Limpieza básica de datos: unificar nombre de columnas

In [ ]:
data['Recuperado'].unique()

In [ ]:
data.loc[data['Recuperado'] == 'fallecido'] = 'Fallecido'

In [ ]:
status_labels = data['Recuperado'].unique()
status_labels

### 4.2. Gráfica de cuenta de valores de la columna

In [ ]:
plt.figure(dpi=300)
ax = sns.countplot(y='Recuperado', ax=plt.gca(), data=data)

### 4.3. Tabla con cuenta de valores de la columna

In [ ]:
data.groupby('Recuperado').describe()

### 4.4. Procesamiento de cuentas por agrupamiento

In [ ]:
recuperado_data = data.groupby('Recuperado')


In [ ]:
num_Activo = len(recuperado_data.groups['Activo'])
num_Fallecido = len(recuperado_data.groups['Fallecido'])
num_Recuperado = len(recuperado_data.groups['Recuperado'])
num_Total = num_Activo + num_Fallecido + num_Recuperado

#### 4.4.1. Organización de datos procesados en una tabla de Pandas

In [ ]:
perc_recuperado = pd.DataFrame({
    'Estado':[
        'Activos',
        'Recuperados',
        'Fallecidos'],
    '%':[ 
        100*num_Activo/num_Total,
        100*num_Recuperado/num_Total,
        100*num_Fallecido/num_Total,
        ],
})
perc_recuperado

#### 4.4.2. Redondeo de datos

In [ ]:
perc_recuperado['%'] = perc_recuperado.apply(lambda row: np.round(row['%'], 1), axis=1)
perc_recuperado

## 5. Análisis de la columna *Edad* para datos Covid19
### 5.1. Limpieza de datos: eliminación de datos inválidos

In [ ]:
for e in data['Edad']:
    try:
        float(e)
    except:
        print(e)

In [ ]:
data['Edad'] = data.apply(lambda row: np.NaN if type(row['Edad']) == type('String') else row['Edad'], axis=1)

In [ ]:
for e in data['Edad']:
    try:
        float(e)
    except:
        print(e)

### 5.2. Diagrama de violín
#### 5.2.1. Todos los datos

In [ ]:
plt.figure(dpi=300)
_ = sns.violinplot(y='Edad', data=data, ax=plt.gca())

#### 5.2.2. Por agrupamientos

In [ ]:
plt.figure(dpi=300)
_ = sns.violinplot(x='Recuperado', y='Edad', data=data, ax=plt.gca())

### 5.3. Gráficos Boxplot
#### 5.2.1. Todos los datos

In [ ]:
plt.figure(dpi=300)
_ = data.boxplot(column='Edad', ax=plt.gca())

#### 5.3.2. Por agrupamientos

In [ ]:
plt.figure(dpi=300)
_ = data.boxplot(column='Edad', by='Recuperado', ax=plt.gca())

## 6. Pruebas de normalidad
Hay diferentes [pruebas de normalidad](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3693611/), en este Jupyter Notebook se aplican tres de ellas.
### 6.1. Limpieza de datos: eliminación de datos no finitos

In [ ]:
mask = np.isfinite(data['Edad'])
data = data.loc[mask]

### 6.2. Prueba de [Kolmogorov–Smirnov](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.kstest.html)
Esta es una prueba no parámetrica para determinar si una cierta muestra sigue una determinada distribución de probabilidad.  Esta prueba cuantifica la distancia entre la función de distribución empírica de una muestra y la función de distribución acumulativa de la distribución de referencia.

La hipótesis nula es que los datos analizados siguen la distribución de referencia. Si $p-value \geq 0.05$, se acepta la hipótesis nula, de lo contrario, se rechaza. 
#### 6.2.1. Para todos los datos

In [ ]:
ks = stats.kstest(data['Edad'],'norm')
print('statiscic: %0.2f, p-value=%0.5g\n' % (ks.statistic, ks.pvalue))

#### 6.2.1. Para cada conjunto agrupados por *Recuperado*

In [ ]:
recuperado_data = data.groupby('Recuperado')
ks_dict = {'status':[], 'statistic':[], 'p':[]} 
for group, this_data in recuperado_data:
    ks = stats.kstest(data['Edad'],'norm')
    ks_dict['status'].append(group)
    ks_dict['statistic'].append(ks.statistic)
    ks_dict['p'].append(ks.pvalue)
ks_df = pd.DataFrame(ks_dict)
ks_df

### 6.2. Prueba [Anderson-Darling](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.anderson.html)
Esta prueba estadística permite deterimar si una muestra sigue una cierta distribución estadística de referencia.  

La hipótesis nula para esta prueba es que los datos corresponden con la distribución de referencia.

Si el estadístico es mayor que los valores críticos para los niveles de significancia, entonces, se dice que la hipótesis nula es rechazada.

#### 6.3.1. Para todos los datos

In [ ]:
ad = stats.anderson(data['Edad'], dist='norm')
ad

#### 6.3.1. Para cada conjunto agrupados por *Recuperado*

In [ ]:
recuperado_data = data.groupby('Recuperado')
ad_dict = {'status':[], 'statistic':[]} 
for group, this_data in recuperado_data:
    ad = stats.anderson(data['Edad'], dist='norm')
    ad_dict['status'].append(group)
    ad_dict['statistic'].append(ad.statistic)
    idx = 0
    for c in ad.critical_values:
        idx += 1
        key = 'Critical Value %d' % idx
        if not key in ad_dict.keys():
            len_status = len(ad_dict['status'])
            ad_dict[key] = [] if len_status == 1 else len_status*[np.NaN]
        ad_dict[key].append(c)
    idx = 0
    for s in ad.significance_level:
        idx += 1
        key = 'Significance Level %d' % idx
        if not key in ad_dict.keys():
            len_status = len(ad_dict['status'])
            ad_dict[key] = [] if len_status == 1 else len_status*[np.NaN]
        ad_dict[key].append(s)

ad_df = pd.DataFrame(ad_dict)
ad_df

### 6.3. Prueba de [Shapiro-Wilk](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.shapiro.html)

Esta prueba estadística permite determinar si los datos siguen una distribución normal.

La hipótesis nula es que los datos siguen la distribución normal.  Si $p-value \geq 0.05$, se acepta la hipótesis nula, de lo contrario, se rechaza.
#### 6.3.1. Para todos los datos

In [ ]:
statistic, p = stats.shapiro(data['Edad'])
print('statiscic: %0.2f, p-value=%0.5g\n' % (statistic, p))

#### 6.3.1. Para cada conjunto agrupados por *Recuperado*

In [ ]:
recuperado_data = data.groupby('Recuperado')
sw_dict = {'status':[], 'statistic':[], 'p':[]} 
for group, this_data in recuperado_data:
    statistic, p = stats.shapiro(data['Edad'])
    sw_dict['status'].append(group)
    sw_dict['statistic'].append(statistic)
    sw_dict['p'].append(p)
sw_df = pd.DataFrame(sw_dict)
sw_df

## 7. Análisis básico de evolución temporal

In [ ]:
time_labels = ['fecha reporte web', 'Fecha de notificación', 'Fecha de inicio de síntomas','Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación']

### 7.1. Limpieza de datos: conversión de formato y remoción de datos inválidos

In [ ]:
time_label='Fecha de diagnóstico'
date_time_str = data[time_label]
for d in date_time_str:
    try:
        datetime. strptime(d, '%d/%m/%Y %H:%M:%S')
    except:
        print(d)
        break

In [ ]:
def to_datetime(row, time_label='Fecha de diagnóstico'):
    d = row[time_label]
    this_data = None
    try:
        this_data = datetime.strptime(d, '%d/%m/%Y %H:%M:%S')
    except:
        return np.NaN
    return this_data

In [ ]:
data[time_label] = data.apply(to_datetime, axis= 1)
mask = ~data[time_label].isna()
data = data.loc[mask]

### 7.2. Línea temporal

In [ ]:
t0 = min(data[time_label])
t_serie = data[time_label] - t0
data['t [días]'] = [d.days for d in t_serie]

### 7.3. Submuestreo y gráfica de la columna *Edad* en función del tiempo
#### 7.3.1. Submuestreo para graficar solo N muestras

In [ ]:
N = 500
data_len = len(data)
k = np.arange(0, data_len, int(data_len/N))

#### 7.3.2. Gráfica para todas las edades

In [ ]:
plt.figure(dpi=300)
plt.plot(data.iloc[k]['t [días]'], data.iloc[k]['Edad'], 'ko')
plt.xlabel('Tiempo [días]')
_ = plt.ylabel('Edad [años]')

#### 7.3.2. Gráfica de edades por agrupamiento por *Recuperado*

In [ ]:
recuperado_data = data.groupby('Recuperado')
plt.figure(dpi=300)
marks = 'ov^>'
idx = -1
for group, this_data in recuperado_data:
    data_len = len(this_data)
    k = np.arange(0, data_len, int(data_len/N))
    idx += 1
    plt.plot(this_data.iloc[k]['t [días]'], this_data.iloc[k]['Edad'], marks[idx], label=group)
plt.xlabel('Tiempo [días]')
plt.ylabel('Edad [años]')
_ = plt.legend()

### 7.4. Medias para cada 30 días
#### 7.4.1. Método para obtener las medias mensuales para la *Edad*

In [ ]:
def get_means(df):
    t2_array = np.arange(1, int(max(df['t [días]']))+30, 30)
    t1_array = t2_array - 1
    t = []
    mean = []
    for t1, t2 in zip(t1_array, t2_array):
        mask = (df['t [días]'] >= t1) & (df['t [días]'] < t2)
        t.append(t1/30)
        mean.append(np.mean(df.loc[mask, 'Edad'].dropna()))
    return t, mean

#### 7.4.2. Método para obtener el número de casos mensuales

In [ ]:
def get_counts(df):
    t2_array = np.arange(1, int(max(df['t [días]']))+30, 30)
    t1_array = t2_array - 1
    t = []
    counts = []
    for t1, t2 in zip(t1_array, t2_array):
        mask = (df['t [días]'] >= t1) & (df['t [días]'] < t2)
        t.append(t1/30)
        counts.append(np.count_nonzero(mask.dropna()))
    return t, counts

#### 7.4.2. Gráfica de media de edades por agrupamiento por *Recuperado*

In [ ]:
idx = -1
plt.figure(dpi=300)
for group, this_data in recuperado_data:
    idx += 1
    t, mean = get_means(this_data)
    plt.plot(t, mean, marks[idx], label=group)
plt.xlabel('Tiempo [meses]')
plt.ylabel('Edad [años]')
_ = plt.legend()

#### 7.4.2. Gráfica de casos por agrupamiento por *Recuperado*

In [ ]:
idx = -1
plt.figure(dpi=300)
for group, this_data in recuperado_data:
    idx += 1
    t, counts = get_counts(this_data)
    plt.semilogy(t, counts, marks[idx], label=group)
plt.xlabel('Tiempo [meses]')
plt.ylabel('Edad [años]')
_ = plt.legend()

## 8. Preguntas de autoexplicación
### 8.1. Primer conjunto de preguntas de auto explicación

8.1.1. En el ítem 1 se importan unas librerías.  Describa, de manera resumida, el propósito de cada librería importada.

8.1.2. En el ítem 2 hay dos métodos para cargar los datos, en el primero se cargan desde una carpeta local, que se crea cuando se [clona el repositorio](https://github.com/davidalejandromiranda/StatisticalPhysics.git), y en el segundo, desde un archivo en la nube.  ¿Cuál es el propósito de utilizar *try*.

8.1.3. En el ítem 3.1 se describen los datos importados, *data.describe()*, ¿de qué manera los valores de las filas *count*, *mean*, *std*, *min*, *25%*, *50%*, *75%* y *max* describen los datos de cada columna? Se sugiere comparar los ítems 3.1 y 3.4.

8.1.4. Los datos importados están organizados por columnas, ¿qué información contienen dichas columnas?

8.1.5. Describa cómo se analizan los datos de las columnas *Recuperado* y *Edad*.  Tenga en cuenta que antes de procesar los datos estos son limpiados y luego se realiza el análisis.

### 8.2. Segundo conjunto de preguntas de auto explicación

8.2.1. Interprete cada una de las pruebas de normalidad aplicadas en el ítem 6.  Se sugiere acompañar la interpretación con los gráficos de violín (ítem 5.2) y los *boxplot* (ítem 5.3) obtenidos.

8.2.2. Describa cómo se realiza la limplieza de datos para el análisis temporal, ítem 7.1.

8.2.3. En la línea 3 del ítem 7.2 se utiliza el comando *d.days*, ¿cuál es el propósito de utilizar dicho comando?

8.2.4. ¿Cuál es el propósito de ralizar un submuestreo de los datos y cómo se realiza?

8.2.5. En el ítem 7.4 se analizan los valores medios de datos muestreados para cada 30 días.  Interprete el análisis temporal de *Edad* teniendo en cuenta el agrupamiento por la columna *Recuperado*.





### 8.3. Tercer conjunto de preguntas de autoexplicación

8.3.1. ¿Es posible obtener una variable aleatoria normal para describir la *Edad*?  Si su respuesta es afirmativa, muestre cómo se hace.

8.3.2. ¿Cuál es la edad media de casos Activos, Fallecidos y Recuperados en los municipios del Área Metropolitana de Bucaramanga?

8.3.3. ¿Cuál ha sido el comportamiento en el tiempo de la edad para los casos Activos, Fallecidos y Recuperados en los municipios del Área Metropolitana de Bucaramanga?

8.3.4. En los datos importados hay más información de la analizada en este ejemplo.  Presente su propio análisis complementario utilizando una o más celdas de una copia de este Jupyter Notebook.

8.3.5. Describa, en sus propias palabras, qué aprendió al resolver las preguntas de autoexplicación de este Jupyter Notebook.

End!